In [2]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

In [3]:
prompt = PromptTemplate(template="write a program about {topic}", input_variables=["topic"])

model = OllamaLLM(model="gemma:2b")
# 或者 model = OllamaLLM(model="gemma:latest")

output_parser = StrOutputParser()

chain = model.generate([prompt.format(topic="讲一个关于软件的笑话")])
print(chain.generations[0][0].text)

```python
# 讲一个关于软件的笑话

def tell_joke():
  # 准备笑话
  joke = "软件总是像一只小狗，它会把它尾巴弄得有点乱了。"
  
  # 播放笑话
  print(joke)


# 运行函数
tell_joke()
```

**运行结果:**

```
软件总是像一只小狗，它会把它尾巴弄得有点乱了。
```

**解释:**

1. `tell_joke` 函数定义了一个 `joke`变量， containing 要讲的笑话。
2. `print(joke)` 语句使用 `print` 函数打印笑话。
3. `tell_joke` 函数调用 `tell_joke` 函数，执行笑话。

**其他选项:**

* 可以修改笑话，使其更有趣或更令人笑。
* 可以添加一些背景信息，让笑话更有意义。
* 可以使用不同的语言编写笑话。


## 使用LCEL


In [4]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [17]:
# 初始化嵌入模型
embedding_model = OllamaEmbeddings(model="gemma:2b")
# 需要被嵌入的文本列表
texts = ["哈里森在肯肖工作", "熊喜欢吃蜂蜜"]
vectorstore = DocArrayInMemorySearch.from_texts(
    texts,
    embedding=embedding_model
)
retriever = vectorstore.as_retriever()
template = """根据以下上下文回答问题: {context} 问题: {question} """
prompt = ChatPromptTemplate.from_template(template)
model = OllamaLLM(model="gemma:2b")
output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser
result = chain.invoke("哈里森在哪里工作？")
print(result)


C:\Software\anaconda\conda\envs\tf20\lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


该上下文没有提供哈里森在哪里工作的具体信息，无法回答该问题。


In [18]:
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama as OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langchain_core.prompts import format_document, ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents.base import Document
from operator import itemgetter

_template = """给定以下对话历史和一个后续问题,请将后续问题转述为一个独立的问题,使用原始语言。

对话历史:

{chat_history}

后续问题: {question}

独立问题:"""

CONDENSE_QUESTION_PROMPT = ChatPromptTemplate.from_template(_template)

template = """基于以下背景信息回答问题:

{context}

问题: {question}

"""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="gemma:2b")

vectorstore = FAISS.from_texts(
    ["Langchain是一个用于开发LLM应用的开源框架。它提供了许多有用的工具和组件,如提示模板、记忆体系、索引等,帮助开发者更轻松地构建LLM应用。"],
    embedding=OllamaEmbeddings(model="gemma:2b")
)

retriever = vectorstore.as_retriever()

_inputs = RunnableParallel(
    chat_history=RunnablePassthrough(),
    question=(
        {"question": RunnablePassthrough(), "chat_history": lambda x: get_buffer_string(x["chat_history"])}
        | CONDENSE_QUESTION_PROMPT
        | model
        | StrOutputParser()
    ),
)

_context = {
    "context": itemgetter("question") | retriever | (lambda docs: format_document(Document(page_content="\n".join(doc.page_content for doc in docs), metadata={"context": "\n".join(doc.page_content for doc in docs), "question": None}), prompt)),
    "question": lambda x: x["question"],
}

conversational_qa_chain = _inputs | _context | prompt | model

chat_history = [
    HumanMessage(content="Langchain是什么?"),
    AIMessage(content="Langchain是一个用于开发大语言模型应用的开源框架。"),
    HumanMessage(content="那么Langchain有哪些主要功能?"),
]

print(conversational_qa_chain.invoke({"question": "Langchain有哪些特点?", "chat_history": chat_history}))

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).